In [7]:
import pandas as pd

frame = pd.read_pickle('/Users/giuliavenditti/Desktop/data.pkl')
frame.head()

,issn,doi,doi-num,on-crossref,reference,asserted-by-cr,asserted-by-pub,ref-undefined,ref-num,year,country,subject
0,2372-0352,10.3934/environsci.2020019,1,1,1,0,52,24,76,2020.0,US,G
1,2372-0352,10.3934/environsci.2020024,1,1,1,0,11,8,19,2020.0,US,G
2,2372-0352,10.3934/environsci.2020026,1,1,1,0,26,23,49,2020.0,US,G
3,2372-0352,10.3934/environsci.2015.3.565,1,1,1,0,37,0,37,2015.0,US,G
4,2372-0352,10.3934/environsci.2021009,1,1,1,21,0,19,40,2021.0,US,G


# 1) DOI on Crossref

In [2]:
import plotly.graph_objects as go
import plotly.express as px

frame = frame.groupby('subject').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_cr'] = (frame['on-crossref']/frame['doi-num'])
fig = px.bar(frame,x=frame.index,y='perc_cr')
fig.show()
# fig.write_html('perc_cr_subj.html')

In [ ]:
frame = frame.groupby('country').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_cr'] = (frame['on-crossref']/frame['doi-num'])
fig = px.bar(frame,x=frame.index,y='perc_cr')
fig.show()

In [4]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 1850)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_cr'] = (frame['on-crossref']/frame['doi-num'])
fig = px.bar(frame,x=frame.index,y='perc_cr')
fig.show()

# 2) reference on crosref doi 

In [ ]:
frame = frame.groupby('subject').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_ref'] = (frame['reference']/frame['on-crossref'])
fig = px.bar(frame,x=frame.index,y='perc_ref')
fig.show()

In [13]:
frame = frame.groupby('country').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_ref'] = (frame['reference']/frame['on-crossref'])
fig = px.bar(frame,x=frame.index,y='perc_ref')
fig.show()

In [ ]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 1850)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_ref'] = (frame['reference']/frame['on-crossref'])
fig = px.bar(frame,x=frame.index,y='perc_ref')
fig.show()

# 3) Reference that have doi specified

In [9]:
import plotly.graph_objects as go
import plotly.express as px

frame = frame.groupby('subject').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_ref_nodoi'] = (frame['ref-undefined']/frame['ref-num'])*100
frame['perc_ref_doi'] = (frame['ref-num']/frame['ref-undefined'])*100
fig  = px.histogram(frame, x=frame.index, y= ['perc_ref_nodoi','perc_ref_doi','ref_num'], histfunc='avg')
fig.show()

ValueError: All arguments should have the same length. The length of argument `y` is 3, whereas the length of  previously-processed arguments ['subject'] is 21